In [2]:
import pandas as pd
import config

from pyecharts.charts import Bar
from pyecharts.charts import Timeline
from pyecharts import options as opts

In [3]:
year_list = list(range(2011, 2023))
df_min_data = pd.DataFrame(columns=['日期', '最高温','最低温', '天气', '风力风向', '城市', '地区'])
for csv_name in list(config.map.keys()):
    df = pd.read_csv(csv_name + '.csv', encoding='utf-8', parse_dates=True)   # 读取csv表
    df = df.dropna()
    df.loc[:, "地区"] = csv_name
    df['最低温'] = df['最低温'].astype('int64')
    df['日期'] = pd.to_datetime(df['日期']).dt.year  # 日期列只保留年份
    
    for year in year_list:

        df_year = df.query('日期=={}'.format(str(year)))
        df_min = df_year.query('最低温=={}'.format(df_year['最低温'].min())).iloc[:1]
        df_min_data = pd.concat([df_min_data, df_min])

In [4]:
color_num = ['#D3D3D3','#EEE8AA','#FAF0E6','#FF0000','#006400','#B22222','#FAEBD7','#800000',\
                '#8FBC8F','#FFF0F5','#F0FFFF','#FF69B4','#778899','#C71585','#3CB371','#D8BFD8',\
                '#F0E68C','#FF00FF','#D2B48C','#8B008B','#FF4500','#BA55D3','#FFC0CB','#9932CC',\
                '#00FF7F','#8A2BE2','#FFEBCD','#7B68EE','#FFDAB9','#483D8B','#FFE4E1','#F8F8FF','#808080','#0000CD']

N = 12
color = [val for val in color_num for _ in range(N)]
df_min_data['color'] = color
df_min_data.head(5)



日期 最高温  最低温    天气     风力风向  城市  地区    color
11307  2011  12    2     阴  无持续风向微风  韶关  广东  #D3D3D3
11456  2012   5    1    小雨  无持续风向微风  韶关  广东  #D3D3D3
11588  2013   5    2    小雨  无持续风向微风  韶关  广东  #D3D3D3
11785  2014   9   -2  多云~晴  无持续风向微风  韶关  广东  #D3D3D3
29804  2015  -1  -19   晴~霾    东北风3级  清远  广东  #D3D3D3

In [10]:

tl = Timeline()
for i in year_list:
    # ascending =是否升序排序，默认升序为True，降序则为False
    df_sub = df_min_data[df_min_data['日期'] == i].sort_values(by='最低温', ascending=False)
    province_list = list(df_sub['地区'])
    value_list = list(df_sub['最低温'])
    color_list = list(df_sub['color'])

    y = []
    for j in range(34):
        y.append(
            opts.BarItem(
                name=province_list[j],
                value=value_list[j],
                itemstyle_opts=opts.ItemStyleOpts(color=color_list[j])
            )
        )
    bar = (
        Bar(init_opts=opts.InitOpts(bg_color='#FFF8D7', width='900px', height='500px', ))
        .add_xaxis(province_list)
        .add_yaxis('最低温度', y, bar_width=10, 
                    label_opts=opts.LabelOpts(position='right'), 
                    markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_='average')]),
                    markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='min')])
                )
        .reversal_axis()
        .set_global_opts(
            title_opts=opts.TitleOpts('各省/市最低温{}年'.format(i), 
                                        pos_left='10%'),
            legend_opts=opts.LegendOpts(pos_left='center'),
            xaxis_opts=opts.AxisOpts(
                name='温度°C',
                type_='value',
                name_location='end',
                name_textstyle_opts=opts.TextStyleOpts(
                    font_family='Times News Roma',
                    font_size=14,
                    color='red'
                ),
                min_=-45,
                max_=15,
                interval=5,
                axislabel_opts=opts.LabelOpts(
                    formatter='{value}',
                    font_size=14
                )
                
            ),
            yaxis_opts = opts.AxisOpts(
                name='地区',
                # name_location='end',
                #  坐标轴名称与轴线之间的距离。
                name_gap=10,
                axislabel_opts=opts.LabelOpts(
                    font_size=9,
                    interval=0
                )
            )
        )
    )
    tl.add(chart=bar, time_point='{}年'.format(i))
    tl.add_schema(
    play_interval=1200,
    symbol_size=[10,10]
    )

tl.render_notebook()

    